In [8]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Wed Jun 24 10:38:04 PDT 2020


In [9]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/india_new_start_date/count_data/']

locations = ['India']

In [10]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,357,iron_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,602,baseline,0.0
4,2020,1_to_4,female,covered,covered,person_time,602,iron_fortification_scale_up,0.0


In [11]:
np.unique(pt['folic_acid_fortification_group'])

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [12]:
np.unique(pt['vitamin_a_fortification_group'])

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

# Vitamin A Fortification Coverage

In [13]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','vitamin_a_fortification_scale_up'])] # look only at baseline and vitamin a scenario
covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

overall = data.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / overall
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                         year                                        
baseline                         2020    5.0  0.239741  0.003375  0.235605   
                                 2021    5.0  0.238906  0.002977  0.234406   
                                 2022    5.0  0.239391  0.003618  0.234845   
                                 2023    5.0  0.238800  0.004695  0.232075   
                                 2024    5.0  0.238983  0.005074  0.230834   
vitamin_a_fortification_scale_up 2020    5.0  0.835980  0.002993  0.832693   
                                 2021    5.0  0.917466  0.002868  0.914659   
                                 2022    5.0  0.922958  0.003201  0.919687   
                                 2023    5.0  0.926137  0.003851  0.920724   
                                 2024    5.0  0.928352  0.005023  0.920346   

                                            25%       50%       75%       max  
scenario                         year                                          
baseline                         2020  0.237439  0.240408  0.240886  0.244368  
                                 2021  0.237326  0.240657  0.240707  0.241435  
                                 2022  0.236622  0.240503  0.241156  0.243828  
                                 2023  0.237145  0.238944  0.241149  0.244687  
                                 2024  0.237474  0.240894  0.242304  0.243410  
vitamin_a_fortification_scale_up 2020  0.834342  0.835459  0.836798  0.840608  
                                 2021  0.914873  0.916875  0.920386  0.920537  
                                 2022  0.920913  0.921397  0.926382  0.926412  
                                 2023  0.924710  0.925551  0.929649  0.930050  
                                 2024  0.927141  0.929393  0.931775  0.933103

## Vitamin A coverage conclusion


- coverage is starting in 2020 still, not 2021, although the coverage value that intervention scenario doesn't make sense to me
- based on the coverage algorithm, the 2021 coverage in the intervention scenario should be 89%, which it is.
- not sure why 2020 intervention scenario coverage in 2020 is 80%... this is outside of the confidence interval for parameter b in the coverage algorithm